In [1]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.5 # 2aB
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto3g", verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Mon Sep 29 00:38:03 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

In [ ]:
# def thouless_trans(t1):
#     # U, s, V = np.linalg.svd(t1.T)
#     # S = np.zeros_like(t1.T, dtype=float)
#     # np.fill_diagonal(S, s)
#     # u_ai = U@S
#     # u_ji = V.T
#     Q, R = np.linalg.qr(t1)
#     u_ai = R.T
#     u_ji = Q
#     u = np.vstack((u_ji,u_ai))
#     return u

In [2]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'rhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [3]:
def thouless_trans(t1):
    q, r = np.linalg.qr(t1)
    u_ai = r.T
    u_ji = q
    u_occ = np.vstack((u_ji,u_ai))
    u, _, _ = np.linalg.svd(u_occ)
    return u
u = thouless_trans(10*mycc.t1)
mo_t = mf.mo_coeff @ u
mf.mo_coeff = mo_t

In [4]:
t2_0 = np.zeros(mycc.t2.shape)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(10*mycc.t1, t2_0, eris)
ecc_t2_0 = eccsd+mf.e_tot
print(ecc_t2_0,eccsd)

-2.7500806685522616 0.00023891840489541436


In [5]:
from ad_afqmc.lno.afqmc import LNOAFQMC
thresh = 1e-10
mfcc = LNOAFQMC(mf, thresh=thresh, frozen=None).set(verbose=5)
mfcc.thresh_occ = thresh
mfcc.thresh_vir = thresh
mfcc.nblocks = 5
mfcc.nwalk_per_proc = 5
mfcc.lo_type = 'pm'
mfcc.no_type = 'cim'
mfcc.frag_lolist = '1o'
mfcc.force_outcore_ao2mo = True
mfcc.kernel()


******** <class 'ad_afqmc.lno.afqmc.afqmc.LNOAFQMC'> ********
nocc = 3, nmo = 6
max_memory 4000 MB (current use 170 MB)
thresh_occ = 1.000000e-10
thresh_vir = 1.000000e-10
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = pm


LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 3  nlo = 3
frag_atmlist = None
frag_lolist = [[0], [1], [2]]
frag_wghtlist = [1. 1. 1.]
frag_nonvlist = None
    CPU time for LO and fragment        0.45 sec, wall time      0.37 sec
ao2mo est mem= 0.01 MB  avail mem= 3818.72 MB
Lov is saved to /tmp/gydz9xzi
    CPU time for transforming DF-MP2 integrals      0.00 sec, wall time      0.00 sec
    CPU time for Integral xform         0.00 sec, wall time      0.02 sec
    CPU time for make pno               0.00 sec, wall time      0.00 sec
Solving impurity problem
#
# Preparing LNO-AFQMC calculation
# Calculating Cholesky integrals
# Generating Cholesky Integrals
# frozen orbitals are 0
# local active orbitals are [0 1 2 3 4 5]
# local active space size 6
# loc_eris shape: (36, 36)
# chol shape: (14, 36)
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number of electrons: (3, 3)
# Number of basis functions: 6
# Number of Cholesky vectors: 

-0.157

In [197]:
# u = lno_maker.thouless_trans(mycc.t1)
# mo_t = mf.mo_coeff @ u
# mf.mo_coeff = mo_t
from ad_afqmc import pyscf_interface, config, mpi_jax, driver
import numpy as np

config.setup_jax()
MPI = config.setup_comm()

pyscf_interface.prep_afqmc(mf,chol_cut=1e-7)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))
# init_walkers = trial.get_init_walkers(wave_data, prop.n_walkers, restricted=True)
# init_walkers = u.T @ init_walkers
# driver.afqmc(ham_data,ham,prop,trial,wave_data,sampler,observable,options,MPI,init_walkers)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (3, 3)
# Number of basis functions: 24
# Number of Cholesky vectors: 94
#
# Number of MPI ranks: 1
#
# norb: 24
# nelec: (3, 3)
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 2
# walker_type: rhf
# trial: rhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [198]:
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)
prop_data['walkers'] = u.T @ prop_data['walkers']
prop_data = prop.init_prop_data(trial, wave_data, ham_data, prop_data['walkers'])
print(prop_data['e_estimate'])
print(mf.e_tot)
print(ecc_t2_0)

-2.975836867725512
-3.0035482561056757
-2.975836864672674


In [199]:
from jax import numpy as jnp
from jax import vmap
def walker_rhf_energy1(walker, ham_data, trial_mo):
    h0, rot_h1, rot_chol = ham_data["h0"], ham_data["rot_h1"], ham_data["rot_chol"]
    ene0 = h0
    green_walker = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    ene1 = 2.0 * jnp.sum(green_walker * rot_h1)
    f = jnp.einsum("gij,jk->gik", rot_chol, green_walker.T, optimize="optimal")
    c = vmap(jnp.trace)(f)
    exc = jnp.sum(vmap(lambda x: x * x.T)(f))
    ene2 = 2.0 * jnp.sum(c * c) - exc
    return ene0+ene1+ene2

def rhf_olp(rhf1,rhf2):
    '''<rhf1|rhf2>'''
    olp = jnp.linalg.det(rhf1.T.conj() @ rhf2) ** 2
    return olp

In [ ]:
def walker_rhf_energy2(walker,ham_data,trial_mo):
    '''rhf walker's energy with non-orthonal trial'''
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    green_walker = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gia,ak->gik',rot_chol[:,:nocc,nocc:],
                   green_walker.T[nocc:,:nocc],optimize='optimal')
    tf = vmap(jnp.trace)(f)
    tc = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    ene11 = 2*jnp.einsum('ia,ia->',green_walker[:nocc,nocc:],rot_h1[:nocc,nocc:])
    ene12 = 2*jnp.sum(tf*tc) \
            - jnp.einsum('gik,gki->',f,rot_chol[:,:nocc,:nocc])
    ene1 = ene11-2*ene12
    ene2 = 2*jnp.sum(tf*tf) \
           - jnp.einsum('gij,gji->',f,f)
    return ene1+ene2,ene1,ene2,

In [ ]:
def walker_norhf_orb_en(walker,ham_data,wave_data):
    '''rhf walker's energy with non-orthonal trial'''
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    nocc = rot_h1.shape[0]
    green_walker = (walker.dot(jnp.linalg.inv(
        wave_data['mo_coeff'].T.conj() @ walker))).T
    f = jnp.einsum('gia,ak->gik',rot_chol[:,:nocc,nocc:],
                   green_walker.T[nocc:,:nocc],optimize='optimal')
    tf = vmap(jnp.trace)(f)
    # tc = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    ene11 = 2*jnp.einsum('ia,ka,ik->',green_walker[:nocc,nocc:],rot_h1[:nocc,nocc:],m)
    ene12 = 2*jnp.einsum('gik,ik,gjj->',f,m,rot_chol[:,:nocc,:nocc]) \
            - jnp.einsum('gij,gjk,ik->',f,rot_chol[:,:nocc,:nocc],m)
    ene1 = ene11-2*ene12
    ene2 = 2*jnp.einsum('gik,ik,g->',f,m,tf) \
           - jnp.einsum('gij,gjk,ik->',f,f,m)
    return ene1+ene2

In [ ]:
def _frg_hf_cr(rot_h1, rot_chol, walker, trial, wave_data):
    '''hf orbital correlation energy multiplies the overlap ratio'''
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    nocc = rot_h1.shape[0]
    _calc_green = wavefunctions.rhf(
        trial.norb,trial.nelec,n_batch=trial.n_batch)._calc_green
    green_walker = _calc_green(walker, wave_data)
    f = jnp.einsum('gij,jk->gik', rot_chol[:,:nocc,nocc:],
                    green_walker.T[nocc:,:nocc], optimize='optimal')
    c = vmap(jnp.trace)(f)
    eneo2Jt = jnp.einsum('Gxk,xk,G->',f,m,c)*2 
    eneo2ext = jnp.einsum('Gxy,Gyk,xk->',f,f,m)
    hf_orb_en = eneo2Jt - eneo2ext
    olp_ratio = _calc_olp_ratio_restricted(walker,wave_data)
    hf_orb_cr = jnp.real(olp_ratio*hf_orb_en)
    return hf_orb_cr, jnp.real(hf_orb_en)

In [201]:
walker = prop_data['walkers'][0]
trial_mo = wave_data['mo_coeff']

In [202]:
walker_rhf_energy1(walker,ham_data,trial_mo)

Array(-2.97583687+0.j, dtype=complex128)

In [ ]:
walker_rhf_energy2(walker,ham_data,trial_mo)\
    + walker_rhf_energy1(trial_mo, ham_data, trial_mo)

Array(-2.97583687+0.j, dtype=complex128)

In [205]:
walker_rhf_energy2(walker,ham_data,trial_mo)

Array(-0.6061242+0.j, dtype=complex128)

In [234]:
def walker_norhf_orb_en(walker,ham_data,wave_data):
    '''rhf walker's energy with non-orthonal trial'''
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    nocc = rot_h1.shape[0]
    green_walker = (walker.dot(jnp.linalg.inv(
        wave_data['mo_coeff'].T.conj() @ walker))).T
    f = jnp.einsum('gia,ak->gik',rot_chol[:,:nocc,nocc:],
                   green_walker.T[nocc:,:nocc],optimize='optimal')
    tf = vmap(jnp.trace)(f)
    # tc = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    ene11 = 2*jnp.einsum('ia,ka,ik->',green_walker[:nocc,nocc:],rot_h1[:nocc,nocc:],m)
    ene12 = 2*jnp.einsum('gik,ik,gjj->',f,m,rot_chol[:,:nocc,:nocc]) \
            - jnp.einsum('gij,gjk,ik->',f,rot_chol[:,:nocc,:nocc],m)
    ene1 = ene11+2*ene12
    ene2 = 2*jnp.einsum('gik,ik,g->',f,m,tf) \
           - jnp.einsum('gij,gjk,ik->',f,f,m)
    return ene1+ene2

In [235]:
from ad_afqmc.lno.cc import LNOCCSD
import sys
from ad_afqmc.lno_ccsd import lno_ccsd
from pyscf.lib import logger
from pyscf import lib
from ad_afqmc import wavefunctions
log = logger.Logger(sys.stdout, 6)

In [241]:
t1 = 10*mycc.t1
t2 = np.zeros_like(mycc.t2)
u = thouless_trans(t1)
mf.mo_coeff = mf.mo_coeff @ u
mfcc = mf
thresh = 0
frozen = None
run_frg_list = [0]
use_df_vecs = False
chol_cut = 1e-7
lo_type = 'boys'
no_type = 'ie'
frag_lolist = '1o'

from pyscf.cc.ccsd import CCSD
from pyscf.ci.cisd import CISD
if isinstance(mfcc, (CCSD, CISD)):
    full_cisd = True
    mf = mfcc._scf
else:
    full_cisd = False
    mf = mfcc

if isinstance(thresh, list):
    thresh_occ, thresh_vir = thresh
else:
    thresh_occ = thresh*10
    thresh_vir = thresh

lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = lo_type
lno_cc.no_type = no_type
lno_cc.frag_lolist = frag_lolist
lno_cc.force_outcore_ao2mo = True

s1e = lno_cc._scf.get_ovlp()
orbactocc = lno_cc.split_mo()[1] # non-localized active occ
orbloc = lno_cc.get_lo(lo_type=lo_type) # localized active occ orbitals
m = orbloc.T @ s1e @ orbactocc
lospanerr = abs((m.T @ m) - np.eye(m.shape[1])).max()
if lospanerr > 1e-10:
    log.error('LOs do not fully span the occupied space! '
                'Max|<occ|LO><LO|occ>| = %e', lospanerr)
    raise RuntimeError
# check 2: Span(LO) == Span(occ)
occspanerr = abs(m @ m.T - np.eye(m.shape[0])).max()
if occspanerr < 1e-10:
    log.info('LOs span exactly the occupied space.')
    if no_type not in ['ir','ie']:
        log.error('"no_type" must be "ir" or "ie".')
        raise ValueError
else:
    log.info('LOs span occupied space plus some virtual space.')

if frag_lolist == '1o':
    log.info('Using single-LO fragment') # this is what we use, every active local occ labels a fragment
    frag_lolist = [[i] for i in range(orbloc.shape[1])]
else: print('Only support single LO fragment!')
nfrag = len(frag_lolist)

orb_e = np.empty(nfrag)

if not (no_type[0] in 'rei' and no_type[1] in 'rei'):
    log.warn('Input no_type "%s" is invalid.', no_type)
    raise ValueError

frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [thresh_occ,thresh_vir]
print(f'# lno thresh {thresh_pno}')

if run_frg_list is None:
    run_frg_list = range(nfrag)

from jax import random
seeds = random.randint(random.PRNGKey(options["seed"]),
                    shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

for ifrag in run_frg_list:
    print(f'\n########### running fragment {ifrag+1} ##########')

    fraglo = frag_lolist[ifrag]
    orbfragloc = orbloc[:,fraglo] # the specific local active occ
    # frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    THRESH_INTERNAL = 1e-10
    frzfrag, orbfrag, can_orbfrag \
        = lno_ccsd.make_lno(lno_cc, orbfragloc, THRESH_INTERNAL, thresh_pno)

    mol = mf.mol
    nocc = mol.nelectron // 2 
    nao = mol.nao
    actfrag = np.array([i for i in range(nao) if i not in frzfrag])
    # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
    actocc = np.array([i for i in range(nocc) if i in actfrag])
    actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
    nactocc = len(actocc)
    nactocc = len(actocc)
    nactvir = len(actvir)
    prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]

    print(f'# active orbitals: {actfrag}')
    print(f'# active occupied orbitals: {actocc}')
    print(f'# active virtual orbitals: {actvir}')
    print(f'# frozen orbitals: {frzfrag}')

    # if full_cisd:
    #     # print('# This method is not size-extensive')
    #     frz_mo_idx = np.where(np.array(frozen_mask) == False)[0]
    #     act_mo_occ = np.array([i for i in range(nocc) if i not in frz_mo_idx])
    #     act_mo_vir = np.array([i for i in range(nocc,nao) if i not in frz_mo_idx])
    #     prj_no2mo = lno_ccsd.no2mo(mf.mo_coeff,s1e,orbfrag)
    #     prj_oo_act = prj_no2mo[np.ix_(act_mo_occ,actocc)]
    #     prj_vv_act = prj_no2mo[np.ix_(act_mo_vir,actvir)]
    #     if isinstance(mfcc, CCSD):
    #         print('# Use full CCSD wavefunction')
    #         print('# Project CC amplitudes from MO to NO')
    #         t1 = mfcc.t1
    #         t2 = mfcc.t2
    #         # project to active no
    #         t1 = lib.einsum("ij,ia,ba->jb",prj_oo_act,t1,prj_vv_act.T)
    #         t2 = lib.einsum("ik,jl,ijab,db,ca->klcd",
    #                 prj_oo_act,prj_oo_act,t2,prj_vv_act.T,prj_vv_act.T)
    #         ci1 = np.array(t1)
    #         ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
    #         ci2 = ci2.transpose(0, 2, 1, 3)
    #     if isinstance(mfcc, CISD):
    #         print('# Use full CISD wavefunction')
    #         print('# Project CI coefficients from MO to NO')
    #         v_ci = mfcc.ci
    #         ci0,ci1,ci2 = mfcc.cisdvec_to_amplitudes(v_ci)
    #         ci1 = ci1/ci0
    #         ci2 = ci2/ci0
    #         ci1 = lib.einsum("ij,ia,ba->jb",prj_oo_act,ci1,prj_vv_act.T)
    #         ci2 = lib.einsum("ik,jl,ijab,db,ca->klcd",
    #                 prj_oo_act,prj_oo_act,ci2,prj_vv_act.T,prj_vv_act.T)
    #         ci2 = ci2.transpose(0, 2, 1, 3)
    #     print('# Finished MO to NO projection')
    #     ecorr_ccsd = '  None  '
    # else:
    #     print('# Solving LNO-CCSD')
    #     ecorr_ccsd,t1,t2 = lno_ccsd.cc_impurity_solve(
    #             mf,orbfrag,orbfragloc,frozen=frzfrag,eris=None
    #             )
    #     ci1 = np.array(t1)
    #     ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
    #     ci2 = ci2.transpose(0, 2, 1, 3)
    #     ecorr_ccsd = f'{ecorr_ccsd:.8f}'
    #     print(f'# lno-ccsd fragment correlation energy: {ecorr_ccsd}')

    nelec_act = nactocc*2
    norb_act = nactocc+nactvir
    
    print(f'# number of active electrons: {nelec_act}')
    print(f'# number of active orbitals: {norb_act}')
    print(f'# number of frozen orbitals: {len(frzfrag)}')

    options["seed"] = seeds[ifrag]
    lno_ccsd.prep_lno_amp_chol_file(
        mf,orbfrag,options,
        norb_act=norb_act,nelec_act=nelec_act,
        prjlo=prjlo,norb_frozen=frzfrag,
        ci1=[],ci2=[],use_df_vecs=use_df_vecs,
        chol_cut=chol_cut,
        )
    
    ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
    lno_ccsd.prep_lnoccsd_afqmc(options))
    trial = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch)

    # ham_data = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
    #                             )._build_measurement_intermediates(ham_data, wave_data)
    ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
    ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)

    prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)
    prop_data['walkers'] = u.T @ prop_data['walkers']
    prop_data = prop.init_prop_data(trial, wave_data, ham_data, prop_data['walkers'])
    print(prop_data['e_estimate'])
    walker = prop_data['walkers'][0]
    orb_e[ifrag] = jnp.real(walker_norhf_orb_en(walker,ham_data,wave_data))

lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 44.086738277603  delta_f= 44.0867  |g|= 8.3512  1 KF 3 Hx
macro= 2  f(x)= 43.523260650787  delta_f= -0.563478  |g|= 9.11184  1 KF 3 Hx
macro= 3  f(x)= 42.924135942392  delta_f= -0.599125  |g|= 9.76965  1 KF 3 Hx
macro= 4  f(x)= 42.29793993884  delta_f= -0.626196  |g|= 10.2892  1 KF 3 Hx
macro= 5  f(x)= 41.653661835892  delta_f= -0.644278  |g|= 10.6625  1 KF 3 Hx
macro= 6  f(x)= 41.000545555308  delta_f= -0.653116  |g|= 10.8838  1 KF 3 Hx
macro= 7  f(x)= 40.347957984851  delta_f= -0.652588  |g|= 10.9497  1 KF 3 Hx
macro= 8  f(x)= 39.70525748885  delta_f= -0.6427  |g|= 10.859  1 KF 3 Hx
macro= 9

In [239]:
sum(orb_e)+walker_rhf_energy1(trial_mo,ham_data,trial_mo)

Array(-2.80901208, dtype=float64)

In [238]:
walker_rhf_energy1(prop_data['walkers'][0],ham_data,trial_mo)

Array(-2.97583726+0.j, dtype=complex128)

In [223]:
walker = prop_data['walkers'][0]

In [ ]:
def walker_rhf_energy2(walker,ham_data,trial_mo):
    '''rhf walker's energy with non-orthonal trial'''
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    green_walker = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gia,ak->gik',rot_chol[:,:nocc,nocc:],
                   green_walker.T[nocc:,:nocc],optimize='optimal')
    tf = vmap(jnp.trace)(f)
    tc = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    ene11 = 2*jnp.einsum('ia,ia->',green_walker[:nocc,nocc:],rot_h1[:nocc,nocc:])
    ene12 = 2*jnp.sum(tf*tc) \
            - jnp.einsum('gik,gki->',f,rot_chol[:,:nocc,:nocc])
    ene1 = ene11+2*ene12 ### bug: a minus sign different from main code
    ene2 = 2*jnp.sum(tf*tf) \
           - jnp.einsum('gij,gji->',f,f)
    return ene1+ene2,ene1,ene2

In [228]:
print(walker_rhf_energy2(trial_mo,ham_data,trial_mo))

(Array(0., dtype=float64), Array(0., dtype=float64), Array(0., dtype=float64))


In [233]:
print(walker_rhf_energy2(walker,ham_data,trial_mo)[0]+walker_rhf_energy1(trial_mo,ham_data,trial_mo))
print(walker_rhf_energy1(walker,ham_data,trial_mo))

(-2.9758368582915393+0j)
(-2.9758368582915393+0j)


In [ ]:
def walker_norhf_orb_en(walker,ham_data,wave_data):
    '''rhf walker's energy with non-orthonal trial'''
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    nocc = rot_h1.shape[0]
    green_walker = (walker.dot(jnp.linalg.inv(
        wave_data['mo_coeff'].T.conj() @ walker))).T
    f = jnp.einsum('gia,ak->gik',rot_chol[:,:nocc,nocc:],
                   green_walker.T[nocc:,:nocc],optimize='optimal')
    tf = vmap(jnp.trace)(f)
    # tc = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    ene11 = 2*jnp.einsum('ia,ka,ik->',green_walker[:nocc,nocc:],rot_h1[:nocc,nocc:],m)
    ene12 = 2*jnp.einsum('gik,ik,gjj->',f,m,rot_chol[:,:nocc,:nocc]) \
            - jnp.einsum('gij,gjk,ik->',f,rot_chol[:,:nocc,:nocc],m)
    ene1 = ene11+2*ene12
    ene2 = 2*jnp.einsum('gik,ik,g->',f,m,tf) \
           - jnp.einsum('gij,gjk,ik->',f,f,m)
    return ene1+ene2

In [179]:
walker = prop_data['walkers'][0]
orb_e = walker_norhf_orb_en(walker,ham_data,wave_data)
print(orb_e)

(0.001425551572255514+0j)


In [ ]:
# ### initialize propagation
# seed = options["seed"]
# propagator = prop
# init_walkers = None
# ham_data = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
#                                 )._build_measurement_intermediates(ham_data, wave_data)
# ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
# ham_data = ham.build_propagation_intermediates(ham_data, propagator, trial, wave_data)

# prop_data = propagator.init_prop_data(trial, wave_data, ham_data, init_walkers)


In [ ]:
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(orbfrag)
# ccsd_corr = mycc.energy(t1,t2,eris)
# print(mf.e_tot+ccsd_corr)
# print(mycc.e_tot)

-5.541455769668331
-5.541455769668331


In [ ]:
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(orbfrag)
# ccsd_corr = mycc.energy(t1,t2_0,eris)
# print(mf.e_tot+ccsd_corr)
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(mycc.mo_coeff)
# ccsd_corr = mycc.energy(mycc.t1,t2_0,eris)
# print(mf.e_tot+ccsd_corr)

-5.370456322958685
-5.3704563229586855
